In [1]:
from movie_lens_lib import *
import pandas as pd
from sklearn.model_selection import train_test_split

## Constants
n_movie_clusters = 5
rating_multiplier = 5
year_multiplier = 0.05
test_sample_size = 100_000
weight_genre, weight_cluster, weight_movie = 0.35, 0.45, 0.2
train_size = 0.9
random_state = 42

## Import & Split dataset
ratings_df = pd.read_csv("data/ratings.csv")
movies_df = pd.read_csv("data/movies.csv", index_col="movieId")

X = ratings_df.drop(["rating"], axis=1)
y = ratings_df["rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)
ratings_train_df = pd.concat([X_train, y_train], axis=1)
ratings_test_df = pd.concat([X_test, y_test], axis=1)

## Preprocess
movies_hot_df = PreProcessingAggregated().transform((movies_df, ratings_train_df))

In [3]:
ratings_test_grouped = ratings_train_df.copy()
ratings_test_grouped["rating_count"] = np.ones(ratings_test_grouped.shape[0])
ratings_test_grouped = ratings_test_grouped.groupby("userId")["rating_count"].sum().to_frame()
ratings_test_df = ratings_test_df.merge(ratings_test_grouped, on="userId")
ratings_test_df.head()

,userId,movieId,timestamp,rating,rating_count
0,23474,8636,1623093417,2.0,158.0
1,23474,593,1622908173,4.5,158.0
2,23474,4963,1622907887,3.5,158.0
3,23474,81845,1622908031,4.0,158.0
4,23474,84152,1622991895,3.5,158.0


In [2]:
results = []

In [ ]:
x_test_sample = ratings_test_df.sample(frac=1, random_state=42)
x_test_sample = x_test_sample[:test_sample_size]
y_true = x_test_sample["rating"].values

hybrid_regressor = HybridRegressor(movies_hot_df).fit(X_train, y_train)
hybrid_predictions = hybrid_regressor.predict(x_test_sample, False)
print_stats(hybrid_results := get_performance_stats(y_true, hybrid_predictions))